## Chapter 5　応用

---
### 5.1　機械翻訳


#### 統計翻訳とニューラル翻訳
ある言語の文章を多言語の文章に翻訳する、人手を介さない方法論を機械翻訳（machine translation）と総称する。  
ルールベース翻訳、用例ベース翻訳、統計翻訳（statistical machine translation）などの方法論があり、  
系列変換モデルを利用した機械翻訳はニューラル翻訳（neural machine translation）と呼ばれる場合が多い。

---
#### NMTの典型的なモデル構成
[OpenNMT](http://opennmt.net/)（旧seq2seq-attn）が現在ベースラインと認識されるモデルの代表である。  
この中のモデルの１つについて取り上げる。

符号化器再帰層に２層bi-directional LSTMを使うとする。  
$\Psi(\cdot)$ をLSTMの処理を表す関数とすると、擬似コードは次の通り。

\begin{align*}
\hline{}
&\text{Input:} \hspace{5pt} X = (x_{i})_{i=1}^{I} \\
&\text{begin} \\
&\hspace{20pt}  h_{0}^{fw1} \leftarrow 0, h_{0}^{fw2} \leftarrow 0, h_{I+1}^{bw1} \leftarrow 0, h_{I+1}^{bw2} \leftarrow 0 \\
&\hspace{20pt}  \bar{X} \leftarrow E^{s}X \\
&\hspace{20pt}  \text{for} \hspace{5pt} i \leftarrow 1 \hspace{5pt} \text{to} \hspace{5pt} I \hspace{5pt} \text{do} \\
&\hspace{40pt} h_{i}^{fw1} \leftarrow \Psi^{fw1}(\bar{x}_{i},h_{i-1}^{fw1}) \\
&\hspace{40pt} h_{i}^{fw2} \leftarrow \Psi^{fw2}(h_{i}^{fw1},h_{i-1}^{fw2}) \\
&\hspace{20pt} \text{end} \\
&\hspace{20pt} \text{for} \hspace{5pt} i \leftarrow 1 \hspace{5pt} \text{to} \hspace{5pt} I \hspace{5pt} \text{do} \\
&\hspace{40pt} i' \leftarrow I+1-i \\
&\hspace{40pt} h_{i'}^{bw1} \leftarrow \Psi^{bw1}(\bar{x}_{i'},h_{i'+1}^{bw1}) \\
&\hspace{40pt} h_{i'}^{bw2} \leftarrow \Psi^{bw2}(h_{i'}^{bw1},h_{i'+1}^{bw2}) \\
&\hspace{40pt} h_{i'}^{s} = h_{i'}^{fw2} + h_{i'}^{bw2} \\
&\hspace{20pt} \text{end} \\
&\text{end} \\
&\text{Output:} \hspace{5pt} H^{s}=(h_{i}^{s})_{i=1}^{I}, h_{I}^{fw1}, h_{I}^{fw2}, h_{1}^{bw1}, h_{1}^{bw2} \\
\hline{}
\end{align*}



次に、復号化再帰層に２層LSTMを使うものとすると、擬似コードは次の通り。

\begin{align*}
\hline{}
&\text{Input:} \hspace{5pt} H^{s}=(h_{i})_{i=1}^{I}, h_{I}^{fw1}, h_{I}^{fw2}, h_{1}^{bw1}, h_{1}^{bw2}, Y=(y_{j})_{j=1}^{J} \\
&\text{begin} \\
&\hspace{20pt} z_{0}^{fw1} \leftarrow h_{I}^{fw1} + h_{1}^{bw1} \\
&\hspace{20pt} z_{0}^{fw2} \leftarrow h_{I}^{fw2} + h_{1}^{bw2} \\
&\hspace{20pt} y_{0} \leftarrow y^{(BOS)}, y_{J+1} \leftarrow y^{(EOS)} \\
&\hspace{20pt} c_{0} \leftarrow 0 \\
&\hspace{20pt} \text{for} \hspace{5pt} j \leftarrow 1 \hspace{5pt} \text{to} \hspace{5pt} J+1 \hspace{5pt} \text{do} \\
&\hspace{40pt} \bar{y}_{j} \leftarrow E^{t}y_{j-1} \\
&\hspace{40pt} z_{j}^{0} \leftarrow \operatorname{concat}(c_{j-1},\bar{y}_{j}) \\
&\hspace{40pt} z_{j}^{fw1} \leftarrow \Psi^{fw1}(z_{j}^{0},z_{j-1}^{fw1}) \\
&\hspace{40pt} z_{j}^{fw2} \leftarrow \Psi^{fw2}(z_{j}^{fw1},z_{j-1}^{fw2}) \\
&\hspace{40pt} h_{j}^{t} \leftarrow z_{j}^{fw2} \\
&\hspace{40pt} a_{j} \leftarrow \operatorname{softmax}((H^{s})^{\mathrm{T}}W_{2}^{a}h_{j}^{t}) \\
&\hspace{40pt} c_{j}' \leftarrow H^{s}a_{j} \\
&\hspace{40pt} \tilde{c}_{j} \leftarrow \operatorname{concat}(c'_{j}, h_{j}^{t}) \\
&\hspace{40pt} c_{j} \leftarrow \operatorname{tanh}(W_{1}^{a} \tilde{c}_{j}) \\
&\hspace{20pt} \text{end} \\
&\hspace{20pt} \text{for} \hspace{5pt} j \leftarrow 1 \hspace{5pt} \text{to} \hspace{5pt} J+1 \hspace{5pt} \text{do} \\
&\hspace{40pt} o_{j} \leftarrow \operatorname{softmax}(W^{O}c_{j}) \\
&\hspace{40pt} \text{loss}_{j} \leftarrow \operatorname{nll}(o_{j}, y_{j}) \\
&\hspace{20pt} \text{end} \\
&\hspace{20pt} \text{loss} \, \leftarrow \sum_{j=1}^{J+1} \text{loss}_{j} \\
&\text{end} \\
&\text{Output:} \hspace{5pt} \text{loss} \\
\hline{}
\end{align*}

評価時の復号化処理は一般にビーム探索を用いて処理を行うことが多い。  
学習後に未知データを評価する際の復号化器における復号化処理を擬似コードで表したものは次の通り。  
（これらのアルゴリズム一式はK=1のビーム探索と等価なアルゴリズムである。）

\begin{align*}
\hline{}
&\text{Input:} \hspace{5pt} H^{s}=(h_{i})_{i=1}^{I}, h_{I}^{fw1}, h_{I}^{fw2}, h_{1}^{bw1}, h_{1}^{bw2} \\
&\text{begin} \\
&\hspace{20pt} z_{0}^{fw1} \leftarrow h_{I}^{fw1} + h_{1}^{bw1} \\
&\hspace{20pt} z_{0}^{fw2} \leftarrow h_{I}^{fw2} + h_{1}^{bw2} \\
&\hspace{20pt} \hat{y}_{0} \leftarrow y^{(BOS)} \\
&\hspace{20pt} c_{0} \leftarrow 0 \\
&\hspace{20pt} j=0 \\
&\hspace{20pt} \text{repeat} \\
&\hspace{40pt} j \leftarrow j+1 \\
&\hspace{40pt} \bar{y}_{j} \leftarrow E^{t}\hat{y}_{j-1} \\
&\hspace{40pt} z_{j}^{0} \leftarrow \operatorname{concat}(c_{j-1},\bar{y}_{j}) \\
&\hspace{40pt} z_{j}^{fw1} \leftarrow \Psi^{fw1}(z_{j}^{0},z_{j-1}^{fw1}) \\
&\hspace{40pt} z_{j}^{fw2} \leftarrow \Psi^{fw2}(z_{j}^{fw1},z_{j-1}^{fw2}) \\
&\hspace{40pt} h_{j}^{t} \leftarrow z_{j}^{fw2} \\
&\hspace{40pt} a_{j} \leftarrow \operatorname{softmax}((H^{s})^{\mathrm{T}}W_{2}^{a}h_{j}^{t}) \\
&\hspace{40pt} c_{j}' \leftarrow H^{s}a_{j} \\
&\hspace{40pt} \tilde{c}_{j} \leftarrow \operatorname{concat}(c'_{j}, h_{j}^{t}) \\
&\hspace{40pt} c_{j} \leftarrow \operatorname{tanh}(W_{1}^{a} \tilde{c}_{j}) \\
&\hspace{40pt} o_{j} \leftarrow W^{O}c_{j} \\
&\hspace{40pt} \hat{y}_{i} \leftarrow \sigma'_{m}(o_{j}) \\
&\hspace{20pt} \text{until} \hspace{5pt} \hat{y}_{j}=y^{(EOS)} \\
&\hspace{20pt} J \leftarrow j-1 \\
&\text{end} \\
&\text{Output:} \hspace{5pt} \hat{Y} = (\hat{y}_{j})_{j=1}^{J} \\
\hline{}
\end{align*}

---
#### 入出力の処理単位、未知語に対する改良
語彙数を増やすと計算量が大きくなりすぎ、また語彙選択が難しくなる。  
一方で未知語を減らすには語彙を増やすしかないが、零にはできない。

SMTの頃より未知語を後処理で何らかの語に置き換えるという手法が一般的である。

また、入出力単位を単語から文字に変更するという手法もあり、  
語彙数の問題を改善できるが、系列長が極めて長くなるという問題もある。

単語と文字の中間的なバイト対符号化（byte pair encoding; BPE）という手法も使われている。  
これは出現頻度順に文字ペアを１つの文字としてまとめ、指定の語彙数まで符号化を進めるというものである。

現時点では、これらの手法の優劣は不明である。

---
#### 被覆に関する改良
NMTの問題として
 - 過剰生成問題（over-generation ploblem） ：同じ単語やフレーズが繰り返し生成される
 - 不足生成問題（under-generation ploblem） ：元の文の必要な語やフレーズが無視される

が知られている。  
これらの問題を解決する手法として、被覆（coverage）という概念の導入が提案されている。

注意機構で計算される注意確率は、復号化器の時刻 $j$ で得られた固定長ベクトルが  
入力分のどの部分と適合するかの確率を計算したものになる。  
この確率の合計が最終的に何らかの値（要素がすべて１のベクトルなど）になるように学習する。

---
### 5.2　文書要約
文書要約問題は、教師あり学習のための正解データが整備されていないという大きな問題がある。

 - 文書要約課題では文書（複数の文）を要約文（複数文）に変換する課題であり、データ整備の負担が大きい。
 - 正解を定義するのが困難で、要約率すらシステムが利用される場面に依存する。
 
といった理由から、整備が十分に行われてこなかったと考えられる。

---
#### 短文生成タスク／見出し生成タスク
文書要約問題の比較的簡易なものとして、  
ニュース記事の１文目を入力として見出しを生成する見出し生成タスク（headline generation task）について  
ニューラルネットによる符号化復号化モデルを用いた文書要約法が2015年に提案されている。

　（A.M.Rush et al, [A Neural Attention Model for Abstractive Sentence Summarization](https://arxiv.org/abs/1509.00685). Proceedings of EMNLP 2015）

見出し生成タスクのための正解入出力ペアは比較的簡単かつ大量に集めることが可能で、  
１文を１文に変換する課題であることからデータ量も対訳と同程度で済む。

上記論文での実装、学習／評価データ作成手順は公開されており、  
生成型文書要約のベンチマークデータとして広く利用されている。

---
#### 文書要約タスクでの符号化復号化方式の発展
(1)注意機構を用いるモデル

A.M.Rush et al(2015)ではFFNNとattentionを組み合わせた符号化器を用いた見出し生成モデルが提案されており、  
Attention Based Summarization と名付けられている。

入力文長 $I$ と出力（要約）文長 $J$ について $I>J$ を仮定する。  
ABSでは文脈の長さを $C$ として、入力文 $\mathbf{X}$ が与えられた時の要約文 $\mathbf{Y}$ の条件付き確率を次のようにモデル化する。

\begin{align*}
P_{abs}(\mathbf{Y}|\mathbf{X}) = \prod_{j=1}^{J+1} P_{abs}(\mathbf{y}_{j} | \mathbf{X}, \mathbf{Y}_{[j-C, j-1]})
\end{align*}

また、$j$ 番目の出力単語に関する条件付き確率を次のように定義する。

\begin{align*}
P_{abs}(\mathbf{y}_{j}|\mathbf{X},\mathbf{Y}_{[j-C, j-1]}) &= \operatorname{softmax}(\tilde{\mathbf{o}}_{j}) \cdot \mathbf{y}_{j} \\
\tilde{\mathbf{o}}_{j} &= \operatorname{nnlm}(\mathbf{Y}_{[j-C,j-1]}) + \operatorname{enc}(\mathbf{X}, \mathbf{Y}_{[j-C,j-1]})
\end{align*}

ここで $\operatorname{nnlm}(\cdot)$ はFFNNと同等のモデルで、出力層のソフトマックス適用前の値であり  
$\operatorname{enc}(\cdot)$ は注意機構を用いて入力文を符号化したベクトルである。

$\operatorname{nnlm}(\cdot)$ はパラメータとして $\mathbf{E}, \mathbf{U}, \mathbf{O}$ をもち、  
出力側の語彙数を $|\mathcal{V}^{t}|$、単語埋め込みベクトルの次元数を $D$ 、隠れ状態ベクトルの次元数を $H$ とすると  
$\mathbf{E} \in \mathbb{R}^{D \times |\mathcal{V}^{t}|}, \mathbf{U} \in \mathbb{R}^{H \times (CD)}, \mathbf{O} \in \mathbb{R}^{|\mathcal{V}^{t}| \times H}$ となる。



---
ここで、 $\operatorname{nnlm}(\cdot)$ の計算は次のような一連の処理である。  
これは $C$ 単語前までの情報である埋め込みベクトルを使い次の単語を予測するニューラル言語モデルに相当する。

入力層の計算  
　出力側の単語に対応する埋め込みベクトル $\mathbf{e}_{k}$ を埋め込み行列 $\mathbf{E}$ から取得する  
　$j-1$ 番目の単語から前 $C$ 個分の埋め込みベクトルを取得する  
   
　隠れ層の計算の入力ベクトルを得るために入力層で得られた埋め込みベクトルを連結して $\tilde{\mathbf{y}}_{j}$ を得る  
　$\tilde{\mathbf{y}}_{j}$ は $j-C$ から $j-1$ までの $C$ 個の $D$ 次元ベクトルを列方向に連結した $CD$ 次元列ベクトル  
 
隠れ層の計算  
　連結したベクトル $\tilde{\mathbf{y}}_{j}$ に対して変換行列 $\mathbf{U}$ をかけ、  
　関数 $\operatorname{tanh}(\cdot)$ により非線形変換を行い $\mathbf{h}_{j}$ を出力する。  
 
出力層の変換  
　$\mathbf{h}_{j}$ を変換行列 $\mathbf{O}$ を用いて線形変換することで、各出力単語に対するスコアを求める  

---
また、$\operatorname{enc}(\cdot)$ の計算式は次の通り。  
ここで、式中の $\mathbf{F} \in \mathbb{R}^{D \times | \mathcal{V}^{s}|}, \mathbf{E}' \in \mathbb{R}^{D \times | \mathcal{V}^{t}|}$ はそれぞれ入力側／出力側に対応する埋め込み行列、  
$\mathbf{O}' \in \mathbb{R}^{|\mathcal{V}^{t}| \times D}$ は出力層の変換行列、  
$\mathbf{P} \in \mathbb{R}^{D \times (CD)}$ は注意確率を計算する際に利用される入力と出力の関連度を表現した変換行列である。  



\begin{align*}
\operatorname{enc}(\mathbf{X}, \mathbf{Y}_{[j-C,j-1]}) &= \mathbf{O}'\bar{\mathbf{X}}\mathbf{p}_{j} \\
\mathbf{p}_{j} &= \operatorname{softmax}(\tilde{\mathbf{X}}^{\mathrm{T}}\mathbf{P}\tilde{\mathbf{y}}'_{j}) \\
\bar{\mathbf{X}} &= [ \bar{\mathbf{x}}_{1}, \ldots, \bar{\mathbf{x}}_{I}] \\
\bar{\mathbf{x}}_{i} &= \sum_{q=i-Q}^{i+Q} \frac{1}{Q} \tilde{\mathbf{x}}_{q} \\
\tilde{\mathbf{y}}'_{j} &= \operatorname{concat} ( \mathbf{e}'_{j-C}, \ldots, \mathbf{e}'_{j-1}) \\
\tilde{\mathbf{X}} &= \mathbf{FX} \\
\mathbf{e}'_{k} &= \mathbf{E}'\mathbf{y}_{k} \hspace{10pt} \forall k \in \{ j-C, \ldots, j-1 \}
\end{align*}


この処理は入出力間の注意機構の計算に相当する。

はじめに、入出力それぞれを埋め込みベクトルに変換して、  
入力側の埋め込みベクトルのリスト $\tilde{\mathbf{X}}$ と、出力側の $k=j-C$ から $k=j-1$ までの埋め込みベクトル $\mathbf{e}'_{k}$ を得る。

出力 $\mathbf{e}_{k}$ 側は $j-1$ 番目から前 $C$ 個分の埋め込みベクトルを連結して $\tilde{\mathbf{y}}'_{j}$ を得る。

次に $\tilde{\mathbf{X}}$ と $\tilde{\mathbf{y}}'_{j}$ を使って、双線形モデルにより注意確率を計算する。

最後に、入力側の行列 $\bar{\mathbf{X}}$ を得られた注意確率 $\mathbf{p}_{j}$ の確率分布に従って線型結合し、  
変換行列 $\mathbf{O}'$ によって線形変換した値が各単語を選択する $\operatorname{enc}(\cdot)$ のスコアとなる。  
ただし $\bar{\mathbf{X}}$ は窓幅 $Q$ にある入力層の単語埋め込みベクトルの平均値ベクトルのリストを行列形式にしたものである。

---
(2)再帰ニューラルネットに基づく拡張  
翻訳タスクと同様に、見出し生成タスクにおいても符号化器／復号化器ともRNNによるモデル化がよいという発表があり、  
見出し生成タスクでも系列変換モデルが用いられるようになった。

---
#### 今後の発展
要約率  
　要約文長の制御は未解決の課題である。  
　事前に与えられた文字数で要約を生成する方法などが提案されている。  

意味表現の利用  
　意味情報を利用する方法も提案されている。  
　Abstract Meaning Representation と呼ばれる符号化器により意味的な情報をグラフ構造で表現し、  
　このグラフ構造を木構造 LSTM によって符号化して利用する手法が提案されている。

コピー機構  
　文書要約タスクでは入力側に出現した単語そのものを出力として利用することが多く、  
　これを「コピー」処理として導入する方法論が提案されている。

---
### 5.3　対話
対話システム（dialog system）は人とコンピュータが会話することを目的としたシステムで、  
主にユーザインターフェースとして開発が進められている。

旅行案内などのタスク志向対話システム個別のサブシステムの組み合わせで構成されてきたが  
ニューラルネットに基づく手法で、人の発話などを入力、応答を出力とするシステムの直接学習がはじまっている。  
2017年初頭ではテキスト化された会話が主な研究対象であるが、  
直接音声翻訳を生成するなどテキスト翻訳を生成しない研究例もある。


---
#### 対話モデル
主なタスクは対話文脈を参照して適切な応答を予測するタスクである。  
話者が変わることを話者交代（turn-taking）、  
複数の話者交代で区切られた発話の系列を対話文脈（context）と呼ぶ。  

最後の話者交代の後の次の話者がコンピュータ（対話システム）という想定で、  
返すべき発話（応答、response）を予測する。  
これにもう一人の話者が応答し、対話システムが次の応答を予測することを繰り返して対話が行われ、  
目的を達成するか達成できないことが分かるまで対話が続く。

系列変換モデルを適用するときは、入力系列：対話文脈、出力系列：応答　とする。  
また話者交代を示すために、発話の最後を示す仮単語を導入したり  
発話の最初に話者役割を表す仮単語を挿入するなどの方法で話者交代を表現する。

別の手法として、話者交代および発話者を積極的にモデル化した手法も提案されている。  

I.V.Serban et al, [Building End-To-End Dialogue Systems Using Generative Hierarchical Neural Network Models](https://arxiv.org/abs/1507.04808) Proceedings of AAAI, p 3776-3783, 2016  
では文脈状態を表現する新たな隠れ状態ベクトルを用意し、  
これを発話単位で更新することで階層的な文脈符号化を行う手法が提案されている。   

J. Li et al, [A Persona-Based Neural Conversation Model](https://arxiv.org/abs/1603.06155) Proceedings of ACL, p 994-1003, 2016  
では訓練データ内の話者をモデル化しており、  
訓練データ中の話者 ID を使って個性（話者の発話特徴）を反映した応答生成を提案している。

H.Ouchi and Y.tsuboi. [Addressee and Response Selection System](https://github.com/hiroki13/response-ranking) Proceedings of EMNLP, p 2133-2143, 2016.  
では対話内の話者をモデル化しており、各話者ごとの隠れ状態ベクトルで発話者の状態を表すことで  
個々の発話者が１つの対話内で一貫した発言をするように工夫している。  
実装上は、階層モデルの文脈表現状態を発話者ごとに分割し  
隠れ状態ベクトルがその発話者による発話だけで動的に更新されるように拡張している。  
本手法では訓練データにない話者も表現することが可能である。

---
#### 対話モデルの発展
系列変換モデルでは多くの文脈で許容されるような短い応答ばかり出してしまうという問題がある。

現在のモデルでは長い文脈を適切に符号化し応答を生成する能力に不足があることも考えられ、  
I.V.Servn et al (2016) では、スコアが最大の応答を返す代わりに  
条件付き確率 $P(R|C)$ に従って復号化器で応答単語をサンプリングした方が多様な応答が得られること、  
ビーム探索や応答単語列全体の大域的なスコアの最大化では応答の多様性が得られないことを報告している。

J.Li et al. [A Diversity-Promoting Objective Function for Neural Conversation Models](https://arxiv.org/abs/1510.03055) In. Proc of NAACL 2016  
では次のような文脈 $C$ と応答 $R$ の相互情報量を最大化する応答を生成することで  
さまざまな応答が生成されることを示した。

\begin{align*}
\log \frac{P(R,C)}{P(R)P(C)}
\end{align*}

ただし相互情報量の最大化と系列変換モデルの結合は難しいため、  
この論文では応答予測モデル $P(R|C)$ で生成した候補 $k$ 個を  
応答から文脈の予測を行う系列変換モデル $P(C|R)$ で並び替えるなどの手法により近似している。

---
#### 対話システムの自動評価
人手評価を代替する自動評価手法は重要であるが、2017年では明らかな解決策はない。

対話タスクは評価という点では翻訳や要約と類似しており、
どれだけ人の応答に似た発話が返せたかを評価することとなる。
しかし、文脈依存であるために対話ログを参照しての自動評価は難しい。

機械翻訳で使われる評価指標のように１つの参照応答との単語の重なりを重視する評価の結果は    
人手によるシステム応答の評価結果とほとんど相関がない。  
（注：翻訳でも性能が一定の水準に達したことにより、BLEEスコアによるモデルの相対評価が不可能となりつつある）

生成された応答の自動評価の難しさを避けるために  
生成を直接行わず複数の応答候補群から正しい参照応答を選択する応答選択タスクによる評価方法も提案されており、  
システムが出力した候補の $k$ 番目以内に正解が含まれる割合（再現率@k）で自動評価することが一般的である。

応答選択タスクは人とシステムとの比較も容易であり、人の再現率は偶然よりかなり高く、  
対話システムを評価するのに有効と考えられている。  
ただし、不正解応答の作り方に課題があり、妥当な応答が不正解となる可能性があるほか、  
文脈を読み取る能力などを評価できるかどうかは明らかでない。

実環境での話者交代は話者同士の相互作用により、対話システムはこのタイミングも予測する必要がある。  
また複数人会話では話者交代はより複雑となり、対話相手を何らかの方法で明示する必要が出てくる。

---
### 5.4　質問応答 （question answering; QA）
